In [3]:
!pip install pythonnet

     ------------------------------------ 284.5/284.5 kB 247.3 kB/s eta 0:00:00
     -------------------------------------- 51.1/51.1 kB 869.1 kB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os import system

import librosa
import IPython.display as ipd

In [31]:
def feature_extraction(file):
    #file = file[0:17]
    file = path+file
    #print(file)
    #print(path)
    audio_signal, sample_rate = librosa.load(file)
    mfcc_features = librosa.feature.mfcc(y = audio_signal, sr = sample_rate,n_mfcc = 40)
    #print(mfcc_features)
    mfcc_scaled_features = np.mean(mfcc_features.T, axis = 0)
    
    return mfcc_scaled_features

def class_name(file):
    
    file_name = os.path.splitext(file)
    class_name = file_name[0][1:4]
    
    return class_name

In [27]:
filename = '[cel][cla]0001__1.wav'
audio_signal, sample_rate = librosa.load(filename)

C:\Users\anann\AppData\Local\Temp\ipykernel_19060\4023326092.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_signal, sample_rate = librosa.load(filename)


FileNotFoundError: [Errno 2] No such file or directory: '[cel][cla]0001__1.wav'

In [32]:
#EXTRACTING MFCC VALUES FOR EACH AND EVERY AUDIO FILE IN THE WHOLE DATASET

from tqdm import tqdm
path = 'D:\Clg_Projects\Music Dataset\\'
audio_files = os.listdir(path)
#print(audio_files)
extracted_features = []
for item in tqdm(audio_files):
    if item.endswith('.wav'):
        data = feature_extraction(item)
        class_labels = class_name(item)
        extracted_features.append([data,class_labels])

100%|██████████████████████████████████████████████████████████████████████████████| 6706/6706 [03:09<00:00, 35.36it/s]


In [33]:
#CONVERTING THE EXTRACTED MFCC FEATURES INTO A DATAFRAME WITH THEIR CORRESPONDING CLASS LABELS
extracted_features_dataframe = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_dataframe.head()

,feature,class
0,"[-405.65878, 135.69807, -28.65655, 8.057272, 1...",cel
1,"[-308.00064, 150.73721, -12.765315, 20.220102,...",cel
2,"[-401.90668, 143.44278, -5.4553065, 24.519606,...",cel
3,"[-402.59015, 122.679085, -22.919243, 31.608364...",cel
4,"[-387.77686, 128.7044, -14.35541, 20.25297, 1....",cel


In [34]:
extracted_features_dataframe.tail()

,feature,class
6700,"[-56.473892, 93.0696, -5.95356, 30.59589, 3.01...",voi
6701,"[-52.635315, 82.0335, -6.5638366, 31.444872, -...",voi
6702,"[-93.49396, 80.49453, 0.70277464, 11.9647875, ...",voi
6703,"[-85.9429, 62.03944, -9.492248, 22.346514, -9....",voi
6704,"[-57.671223, 83.67719, -23.061024, 24.121489, ...",voi


In [35]:
#SPLITTING THE DATAFRAME INTO DEPENDENT AND INDEPENDENT VARIBALES AND CONVERTING THEM INTO AN ARRAY
x = np.array(extracted_features_dataframe['feature'].tolist())
y = np.array(extracted_features_dataframe['class'].tolist())

In [36]:
x.shape

(6705, 40)

In [37]:
#CONVERTING THE ARRAY 'y'(CATEGORICAL VARIABLE) INTO DUMMY VARIABLES
y = np.array(pd.get_dummies(y))
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [38]:
y.shape

(6705, 11)

In [39]:
#SPLITTING THE DATASET INTO TRAINING AND TESTING SET
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [42]:
!pip install tensorflow

  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
     -------------------------------------- 272.8/272.8 MB 1.5 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
     -------------------------------------- 422.5/422.5 kB 1.0 MB/s eta 0:00:00
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
     ---------------------------------------- 4.2/4.2 MB 1.4 MB/s eta 0:00:00
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
     ---------------------------------------- 1.3/1.3 MB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build depen

In [44]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [48]:
#Number of classes in y
num_labels = y.shape[1]

In [50]:
x_train.shape

(5364, 40)

In [52]:
#SEQUENTIAL MODEL (ATTEMPT 1)
model = Sequential()

#FIRST LAYER
model.add(Dense(100,input_shape = (40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#SECOND LAYER
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#THIRD LAYER
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#FINAL LAYER
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               4100      
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               20200     
                                                                 
 activation_2 (Activation)   (None, 200)               0         
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 100)              

In [55]:
model.compile(loss = 'categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [67]:
#TRAINING A MODEL(ATTEMPT 1)
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 250
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = 'D:\Clg_Projects\music_instruments.hdf5',verbose = 1, save_best_only = True)

start = datetime.now()
model.fit(x_train,y_train,batch_size = num_batch_size,epochs = num_epochs, validation_data = (x_test,y_test),callbacks = [checkpointer])

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/250
138/168 [=======================>......] - ETA: 0s - loss: 1.5848 - accuracy: 0.4703
Epoch 1: val_loss improved from inf to 1.64578, saving model to D:\Clg_Projects\music_instruments.hdf5
168/168 [==============================] - 1s 6ms/step - loss: 1.5776 - accuracy: 0.4718 - val_loss: 1.6458 - val_accuracy: 0.4825
Epoch 2/250
167/168 [============================>.] - ETA: 0s - loss: 1.6113 - accuracy: 0.4577
Epoch 2: val_loss did not improve from 1.64578
168/168 [==============================] - 0s 2ms/step - loss: 1.6098 - accuracy: 0.4577 - val_loss: 1.6766 - val_accuracy: 0.4676
Epoch 3/250
166/168 [============================>.] - ETA: 0s - loss: 1.5846 - accuracy: 0.4710
Epoch 3: val_loss did not improve from 1.64578
168/168 [==============================] - 0s 2ms/step - loss: 1.5839 - accuracy: 0.4713 - val_loss: 1.6676 - val_accuracy: 0.4780
Epoch 4/250
152/168 [==========================>...] - ETA: 0s - loss: 1.5692 - accuracy: 0.4683
Epoch 4: val_loss impr

In [68]:
test_accuracy = model.evaluate(x_test,y_test,verbose = 0)
test_accuracy[1]

0.4839671850204468

In [86]:
#MODEL AND TRAINING (ATTEMPT 2)

#Defining the neural network architecture
#model = tf.keras.

model = Sequential()

#FIRST LAYER
model.add(Dense(100,input_shape = (40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#SECOND LAYER
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#THIRD LAYER
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#FINAL LAYER
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [87]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 100)               4100      
                                                                 
 activation_13 (Activation)  (None, 100)               0         
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_14 (Dense)            (None, 128)               12928     
                                                                 
 activation_14 (Activation)  (None, 128)               0         
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 100)              

In [96]:
model.compile(loss = 'sparse_categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [74]:
#TRAINING A MODEL(ATTEMPT 2)
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = 'D:\Clg_Projects\music_instruments(attempt2).hdf5',verbose = 1, save_best_only = True)

start = datetime.now()
model.fit(x_train,y_train,batch_size = num_batch_size,epochs = num_epochs, validation_data = (x_test,y_test),callbacks = [checkpointer])

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
164/168 [============================>.] - ETA: 0s - loss: 1.6560 - accuracy: 0.4304
Epoch 1: val_loss improved from inf to 1.74193, saving model to D:\Clg_Projects\music_instruments(attempt2).hdf5
168/168 [==============================] - 1s 4ms/step - loss: 1.6551 - accuracy: 0.4312 - val_loss: 1.7419 - val_accuracy: 0.4273
Epoch 2/100
166/168 [============================>.] - ETA: 0s - loss: 1.6478 - accuracy: 0.4337
Epoch 2: val_loss improved from 1.74193 to 1.72286, saving model to D:\Clg_Projects\music_instruments(attempt2).hdf5
168/168 [==============================] - 0s 3ms/step - loss: 1.6482 - accuracy: 0.4338 - val_loss: 1.7229 - val_accuracy: 0.4295
Epoch 3/100
167/168 [============================>.] - ETA: 0s - loss: 1.6438 - accuracy: 0.4366
Epoch 3: val_loss did not improve from 1.72286
168/168 [==============================] - 0s 2ms/step - loss: 1.6465 - accuracy: 0.4355 - val_loss: 1.7494 - val_accuracy: 0.4079
Epoch 4/100
144/168 [==================

In [77]:
x_train

array([[-3.1344073e+02,  1.3957828e+02, -3.2532764e+01, ...,
        -2.4461564e-01, -5.1348501e-01, -1.7173027e+01],
       [-3.6271506e+02,  1.3869600e+02, -6.3733017e+01, ...,
         4.2085199e+00,  1.8003825e+00, -5.7004869e-01],
       [-1.8562082e+02,  9.6106987e+01, -6.0718140e+01, ...,
        -9.4017908e-02, -7.3232036e+00, -2.4550588e+00],
       ...,
       [-1.5897318e+02,  9.0076683e+01, -2.6815508e+01, ...,
         2.3733896e-01, -8.9178152e+00, -2.2492950e+00],
       [-4.5192392e+02,  1.2029269e+02, -2.6154728e+01, ...,
         5.7742944e+00,  8.9114523e+00, -6.4587512e+00],
       [-2.0438811e+02,  1.0340012e+02, -1.3882885e+01, ...,
        -1.2050216e+01, -6.8083439e+00, -2.3715210e+00]], dtype=float32)

In [78]:
y_train

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [79]:
#NORMALIZING THE INPUT FEATURES
x_train = x_train/255.0
x_test = x_test/255.0

In [80]:
#Convert the instrument labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [91]:
y_test

array([[[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]], dtype=float32)

In [97]:
#TRAINING A MODEL(ATTEMPT 4)
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = 'D:\Clg_Projects\music_instruments(attempt3).hdf5',verbose = 1, save_best_only = True)

start = datetime.now()
model.fit(x_train,y_train,batch_size = num_batch_size,epochs = num_epochs, validation_data = (x_test,y_test),callbacks = [checkpointer])

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\anann\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 605, in update_state
        metric_obj.updat

In [92]:
x_train.shape

(5364, 40)

In [93]:
y_train.shape

(5364, 11, 2)

TypeError: only size-1 arrays can be converted to Python scalars